<a href="https://www.kaggle.com/code/zenobiadmello/tutobot?scriptVersionId=280022809" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<img src="https://rohitconsultants.com/imgs/tutobot_logo_1200675.png" width="1200" height="675">


# TutoBot 🧑‍🏫🤖  
### An AI Lesson Planning Agent for Retired Teachers & Neighborhood Tutors

---

## 1. Problem Statement

Many retired teachers and neighborhood tutors struggle to design structured, modern lesson plans, assessments, and learning paths for students — especially as educational standards evolve and expectations shift.

They:
- Rely on outdated notes or ad-hoc planning  
- Feel overwhelmed by “ed-tech” tools  
- Prefer simple, printable materials  

---

## 2. TutoBot — Concept

**TutoBot** is an AI agent system that:

- Understands **subject, grade level, and learning goals**
- Autonomously generates:
  - Multi-week curriculum outlines  
  - Detailed lesson plans  
  - Worksheets & activities  
  - Assessments & rubrics  
- Adapts to **student progress** over time  

The design is **elderly-first**:
- Simple input form (or voice, in future)
- Clear, printable outputs
- Minimal on-screen complexity

---

## 3. Agents Intensive Capstone Alignment

This project is designed to align with the Agents Intensive Capstone:

- ✅ Planner Agent — breaks goals into a structured curriculum  
- ✅ Lesson Agent — generates weekly/daily lesson plans & worksheets  
- ✅ Assessment Agent — creates quizzes & rubrics  
- ✅ Evaluator Agent — reviews quality & alignment  
- ✅ Memory — stores teacher/student context and prior plans (simulated Google Sheets here, real Sheets in production)  
- ✅ Tool Use — (future) integration with Google Sheets / Docs / Forms as external tools  
